In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
from scipy.stats import norm
import time as ttt
import iisignature as iisig
from tqdm import *

In [2]:
steps = 1000
segs = 10
n_paths = 10000
dim = 1

In [3]:
x0 = 100.0 # initial condition
sigma = 0.15 # volatility
r = 0.05 # risk free rate
Strike= 100.0 # strike price
T = 1 # maturity
dt = T/steps # mesh size

In [4]:
def generate_t(T, batch_size = 1):
    '''
    generate time steps
    dim = batch_size * (steps+1). eg. 2000 * 101
    '''
    return np.tile(np.linspace(1e-8, T+1e-8, steps + 1, dtype = np.float32), (batch_size,1))

In [5]:
t = generate_t(T, steps)[0][0:]
len(t)

1001

In [6]:
# Create GBM Paths from Brownian Motion Paths.
def Create_Paths(seed = 666):
    '''
    input: seed, n_paths, number of segments
    output: (1) sigatures: 
                dim = (n_paths * (segments+1) * sig_dim), 
            (2) dW_new: new dW of Y_t
                dim = (n_paths * segments)
            (3) x_T: terminal point of the original paths. (This is not needed, modify later)
                dim = (n_paths * 1)
            (5) paths: original geometric BM paths.
                dim = (n_paths * (steps+1))
            (4) dW: increments of BM of the original paths.
                dim = (n_paths * steps)
    '''

    
    np.random.seed(seed)
    dW = np.sqrt(dt)*np.random.normal(size=(n_paths, dim, steps))
    W = np.concatenate((np.tile(1e-6, (n_paths, dim, 1)), np.cumsum(dW, axis = -1, dtype = np.float32)), axis = -1)

    
    paths = np.zeros([n_paths, dim, steps + 1])
    paths[:, :, 0] = np.ones([n_paths, dim]) * x0
    factor = np.exp((r-(sigma**2)/2)*dt)
    
    # This step generates geometric BM paths. Euler scheme.
    for k in tqdm(range(steps)):
        paths[:, :, k+1] = paths[:, :, k] * (factor * np.exp(sigma * np.squeeze(W[:,:, k+1:k+2] - W[:,:, k:k+1], -1))) 
    paths = np.array(paths, dtype=np.float32)  
    terminal = np.maximum( np.prod(paths[:, :, -1], 1)**(1/dim) - Strike, 0)
    return {'paths': paths, 'W': W, 'terminal': terminal}

In [7]:
Paths_and_W = Create_Paths()
Paths, W, terminal = Paths_and_W['paths'], Paths_and_W['W'], Paths_and_W['terminal']

100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 5478.60it/s]


In [8]:
np.save("Paths_{}_dim_{}_r_{}_sigma_{}_x_{}_K_{}.npy".format(steps, dim, r, sigma, x0, Strike), Paths)
np.save("W_{}_dim_{}.npy".format(steps, dim), W)
np.save("terminal_{}_dim_{}_r_{}_sigma_{}_x_{}_K_{}.npy".format(steps, dim, r, sigma, x0, Strike), terminal)

In [9]:
shrink = int(steps/segs) # eg. shrink steps to segments
dt_new = T/segs # new mesh after shrinkage
level = 2 # truncation level
sig_dim = iisig.siglength(dim+1,level)# dimension of siganature

In [10]:
def Create_Signatures(paths, segments = segs):

    # Initialization
    n_paths = paths.shape[0]
    sig = np.zeros((n_paths, segments + 1, sig_dim))
    t = generate_t(T, steps)[0, None, :]
    
    # This step finds the signature at time 0 = x_0
    path_cut = paths[:,:,0] # cut paths into segments
    time_cut = t[:, 0]  # cut time into segments
    s = iisig.prepare(dim+1, level)

    for i in range(n_paths): 
        stream = np.hstack((time_cut.reshape((-1,1)), path_cut[i].reshape((-1,dim))))  #
        sig[i, 0, :] = iisig.sig(stream, level) # dim = (84, )
        # add path cut end point to sig
    
    # This step computes signatures.
    for k in tqdm(range(segments)):
        path_cut = paths[:, :, :(k+1)*shrink+1]
        time_cut = t[:, :(k+1)*shrink+1]
        for i in range(n_paths):
            stack = np.vstack((time_cut, path_cut[i]))
            stream = stack.T
            sig[i, k+1, :] = iisig.sig(stream, level)
            
    return sig

In [11]:
print("Creating Sig and Log-Sig with {} segments from Paths with {} steps... ".format(segs, steps))
sig = Create_Signatures(Paths, segments = segs)
np.save("sig_{}_from_{}_dim_{}_r_{}_sigma_{}_x_{}_K_{}.npy".format(segs,  steps, dim,r, sigma, x0, Strike), sig)

Creating Sig and Log-Sig with 10 segments from Paths with 1000 steps... 


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.98it/s]


In [12]:
# Selection of W from W
W_full= np.load("W_{}_dim_{}.npy".format(steps, dim))
selection = np.linspace(0,steps, segs+1, dtype = np.int)
W_segs = W_full[:,:, selection]
np.save("W_{}_from_{}_dim_{}.npy".format(segs, steps, dim), W_segs)
Path_segs = Paths[:,:,selection]
np.save("Paths_{}_from_{}_dim_{}_r_{}_sigma_{}_x_{}_K_{}.npy".format(segs, steps, dim, r, sigma, x0, Strike), Path_segs)